# Notebook pour le calcul des indices NDWI, LST (MODIS pour fusion VIIRS)
Prétraitement des produits MODIS LST (MYD21A1D) pour la fusion avec produit VIIRS LST (VNP21A1D)

### Initialisation contexte test appli

In [1]:
import os

# Manually filling Config dictionnary (can be filled by reading Config_process.env) :

CONFIG = {}

CONFIG['WRK_DIR'] = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil/EO4DM')
os.chdir(CONFIG['WRK_DIR'])
CONFIG['WRK_DIR'] = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil')
CONFIG['DATA_HISTO'] = os.path.join(CONFIG['WRK_DIR'],'DATA_HISTO')
CONFIG['ANNEX_DIR'] = os.path.join(CONFIG['WRK_DIR'],'ANNEX')

CONFIG['TERRITORY'] = "New Caledonia (Fr)"
CONFIG['PERIOD_START'] = '2023-11-11'
CONFIG['PERIOD_END'] = '2023-11-21'
CONFIG['DRIVE_FOLDER'] = 'EO4DM_EXPORT_NOTEBOOK'
CONFIG['LANDMASK_ROI'] = 0                                # if 1, preproc products are masked for pixels outside landmask roi (input shapefile to upload on gee asset)
CONFIG['ASSET_EXPORT_L'] = 0                              # if 1, landsat preproc products are exported to gee
CONFIG['ASSET_EXPORT_S2'] = 0                             # if 1, s2 preproc products are exported to gee
CONFIG['ASSET_EXPORT_MOD'] = 0                            # if 1, modis preproc products are exported to gee
CONFIG['CLEAN_GEEFOLDER'] = 0                             # if 1, gee already existing folder is cleaned (deleted, and re-created)
CONFIG['CLEAN_GEECOL'] = 0                                # if 1, gee already exported products are cleaned (deleted, and re-created)
CONFIG['CLEAN_RUNFOLDER'] = 0                             # if 1, drought already existing folder is cleaned (deleted, and re-created)
CONFIG['MODE'] = 'INDICES'                                # 'AUTO', 'MANUAL', 'INDICES' or 'DROUGHT' mode of drought processing chain

CONFIG['lon_min_modis'] = 163.46601498126984
CONFIG['lon_max_modis'] = 168.29999935626984
CONFIG['lat_min_modis'] = -22.85859364271164
CONFIG['lat_max_modis'] = -19.44843739271164


### Création arborescence dossiers DATA et controle ANNEX_DIR

In [2]:
import dmpipeline.DROUGHT_Processing.DROUGHT_global_functions as dglob
OUTDIR_PATHS = dglob.prepare_RUNFolders(CONFIG)

### Test Extrait collections (mois m et décade d)

In [3]:
import ee
import dmpipeline.GEE_Processing.gee_accounts as geeauth
import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

d = 1 # Select decade (0, 1, 2)

PERIOD_START = CONFIG['PERIOD_START']
PERIOD_END = CONFIG['PERIOD_END']
TERRITORY = CONFIG['TERRITORY']
TERRITORY_str = TERRITORY.replace('"', '').replace(' ', '_').replace('(', '').replace(')', '')

# --- GEE Authentification/Initialization ---
path2key = os.path.dirname(geeauth.__file__)
project_id = geegen.googleAuthentification(path2key)
gee_workdir = 'projects'+'/'+project_id+'/'+'assets'

# --- Load ESA Land Cover collection and extract permanent LAND MASK ---
esa_landc_2020 = ee.ImageCollection("ESA/WorldCover/v100").first()
esa_landc_2021 = ee.ImageCollection("ESA/WorldCover/v200").first()
landmask_2020 = esa_landc_2020.neq(80)
landmask_2021 = esa_landc_2021.neq(80)
landmask = landmask_2020.And(landmask_2021)

# --- Load and Filter VIIRS Collections to the specific AREA (territory) and PERIOD (if needed) ---
roi = ee.FeatureCollection("USDOS/LSIB/2017").filter(ee.Filter.stringContains('COUNTRY_NA', TERRITORY))
LSTaqua_dataset = ee.ImageCollection('MODIS/061/MYD21A1D').filterBounds(roi)
REFLECTterra_dataset = ee.ImageCollection('MODIS/061/MOD09GA').filterBounds(roi) # used for extracting scale_out

if PERIOD_START!=[''] and PERIOD_END!=['']:
  LSTaqua_dataset = LSTaqua_dataset.filterDate(PERIOD_START, PERIOD_END)
  REFLECTterra_dataset = REFLECTterra_dataset.filterDate(PERIOD_START, PERIOD_END)
  ALL_dataset = LSTaqua_dataset.merge(REFLECTterra_dataset)
else:
  ALL_dataset = LSTaqua_dataset.merge(REFLECTterra_dataset)
  PERIOD_START = geegen.googleErrorsControl(ALL_dataset.limit(1, 'system:time_start', True).first().date().format('YYYY-MM-dd'), path2key)
  PERIOD_END = geegen.googleErrorsControl(ALL_dataset.limit(1, 'system:time_start', False).first().date().advance(1, 'day').format('YYYY-MM-dd'), path2key)

CRS_OUT = 'EPSG:4326'
GRID_OUT = ee.Geometry.Polygon([[[float(CONFIG['lon_min_modis']), float(CONFIG['lat_max_modis'])],
                                  [float(CONFIG['lon_min_modis']), float(CONFIG['lat_min_modis'])],
                                  [float(CONFIG['lon_max_modis']), float(CONFIG['lat_min_modis'])],
                                  [float(CONFIG['lon_max_modis']), float(CONFIG['lat_max_modis'])]]],None, False)
SCALE_OUT = REFLECTterra_dataset.first().select('sur_refl_b02').projection().nominalScale()

date_start = ee.Date(PERIOD_START)
y = ee.Date(date_start).get('year').getInfo()
m = ee.Date(date_start).get('month').getInfo()
month2find = '{}{:02d}'.format(y,m)

start_D = [1, 11, 21]
end_D = [10, 20, 31]
indicesD_preproc = []
date_start_str = '{}{:02d}{:02d}'.format(y,m,start_D[d])
date_end_str = '{}{:02d}{:02d}'.format(y,m,end_D[d])

LSTaqua_D = (LSTaqua_dataset
            .filter(ee.Filter.calendarRange(y,y,'year'))
            .filter(ee.Filter.calendarRange(m,m,'month'))
            .filter(ee.Filter.calendarRange(start_D[d],end_D[d],'DAY_OF_MONTH')))
REFLECTterra_D = (REFLECTterra_dataset
            .filter(ee.Filter.calendarRange(y,y,'year'))
            .filter(ee.Filter.calendarRange(m,m,'month'))
            .filter(ee.Filter.calendarRange(start_D[d],end_D[d],'DAY_OF_MONTH')))

NbLST_D = geegen.googleErrorsControl(LSTaqua_D.size(), path2key)
NbREFLECT_D = geegen.googleErrorsControl(REFLECTterra_D.size(), path2key)

print(f'NbLST_D = {NbLST_D}')
print(f'NbREFLECT_D = {NbREFLECT_D}')


NbLST_D = 10
NbREFLECT_D = 10


### Test PRETRAITEMENT / COMPOSITION DECADE

In [4]:
import ee
import dmpipeline.GEE_Processing.GEE_generic_functions as geegen
import dmpipeline.GEE_Processing.GEE_preprocessing_functions as geeprep
import dmpipeline.GEE_Processing.GEE_compositing_functions as geecomp

new_collection=[]
QA_dict = []
ASSET_EXPORT_MOD = int(CONFIG['ASSET_EXPORT_MOD'])

#--- LOOP OVER DAYS (in decade D) ---
ALL_dataset_D = LSTaqua_D.merge(REFLECTterra_D)
date_start_D = ALL_dataset_D.limit(1, 'system:time_start', True).first().date()
date_end_D = ALL_dataset_D.limit(1, 'system:time_start', False).first().date()
days_D = geegen.googleErrorsControl(ee.List.sequence(date_start_D.get('day'), date_end_D.get('day')), path2key)

date_start_Dtmp = date_start_D.format('YYYY-MM-dd').getInfo()
date_end_Dtmp = date_end_D.format('YYYY-MM-dd').getInfo()
print(f'date_start_D = {date_start_Dtmp}')
print(f'date_end_D = {date_end_Dtmp}')

for day in days_D:
    lst = LSTaqua_D.filter(ee.Filter.calendarRange(day,day,'DAY_OF_MONTH')).first()
    reflect = REFLECTterra_D.filter(ee.Filter.calendarRange(day,day,'DAY_OF_MONTH')).first()

    indices_preproc, QA_dict  = geeprep.preprocessingMODISv21A1D(lst, reflect, landmask, new_collection, QA_dict, CRS_OUT, GRID_OUT, SCALE_OUT, path2key, ASSET_EXPORT_MOD)
    if indices_preproc=='not considered': pass
    else:
        indicesD_preproc += [indices_preproc]
    del lst, reflect

# --- COMPOSITING ---
indicesD_preproc = ee.ImageCollection(indicesD_preproc)
Nbgoodimages_D = geegen.googleErrorsControl(indicesD_preproc.size(), path2key)
if Nbgoodimages_D==0:
    pass
elif Nbgoodimages_D==1:
    indicesD_comp = {}
    indicesD_comp['LST'] =  indicesD_preproc.first().select('LST')
    indicesD_comp['NDWI'] =  indicesD_preproc.first().select('NDWI')
    nanscores_D = geegen.computeNanScores(indicesD_preproc.first(), path2key, landmask, scale=SCALE_OUT)
    comp_type_D = f'COMPD{d+1}'
elif Nbgoodimages_D > 1:
    indicesD_comp = geecomp.extractComposite(indicesD_preproc, path2key, GRID_OUT)
    nanscores_D = geegen.computeNanScores(indicesD_comp['LST'].addBands(indicesD_comp['NDWI'], overwrite=True).select(['LST', 'NDWI']), path2key, landmask, scale=SCALE_OUT)
    comp_type_D = f'COMPD{d+1}'
del indicesD_preproc


date_start_D = 2023-11-11
date_end_D = 2023-11-20


06/02/2024 12:08:18 - INFO - MODIS 2023-11-11 : preprocessing product
06/02/2024 12:08:33 - INFO - MODIS 2023-11-12 : preprocessing product
06/02/2024 12:08:46 - INFO - MODIS 2023-11-13 : preprocessing product
06/02/2024 12:08:59 - INFO - MODIS 2023-11-14 : preprocessing product
06/02/2024 12:09:14 - INFO - MODIS 2023-11-15 : preprocessing product
06/02/2024 12:09:25 - INFO - MODIS 2023-11-16 : preprocessing product
06/02/2024 12:09:38 - INFO - MODIS 2023-11-17 : preprocessing product
06/02/2024 12:09:51 - INFO - MODIS 2023-11-18 : preprocessing product
06/02/2024 12:10:05 - INFO - MODIS 2023-11-19 : preprocessing product
06/02/2024 12:10:19 - INFO - MODIS 2023-11-20 : preprocessing product


### Export des INDICES COMPOSITES

In [5]:
import time

DRIVE_FOLDER = CONFIG['DRIVE_FOLDER']

# --- EXPORTING RASTERS ---
comp_lst_filename = f'MODISv21A1D_AQUA_FILT_LST_{month2find}_{comp_type_D}'

LST_comp = indicesD_comp['LST']

start_time = time.time()
geegen.exportImage(DRIVE_FOLDER, LST_comp, comp_lst_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=CRS_OUT, data_scale=SCALE_OUT, data_region=GRID_OUT)
elapsed_time = round(time.time() - start_time)

COMP_dict = [ee.Feature(None, {'DATE': month2find,
                               'COMPOSITE': comp_type_D,
                               'NB LST': NbLST_D,
                               'NB REFLECT': NbREFLECT_D,
                               'COMPOSITE NAN SCORE LST': nanscores_D['LST'],
                               'COMPOSITE NAN SCORE NDWI': nanscores_D['NDWI'],
                               'COMPOSITE TIME (sec)': elapsed_time})]

# --- SAVE TABLES (DATA FRAMES) INTO CSV FILES (LOCAL MACHINE) ---
QAtable_filename = f'GEEPREPROC_MODISv21A1D_FILT_{TERRITORY_str}_{date_start_str}_{date_end_str}'
QAtable_columns = ['DATE','NAN SCORE LST','NAN SCORE NDWI','PREPROC TIME (sec)']
COMPtable_filename = f'COUNTCompositeDecade_MODISv21A1D_{TERRITORY_str}_{date_start_str}_{date_end_str}'
COMPtable_columns = ['DATE','COMPOSITE','AquaTerra LST','Terra REFLECT','COMPOSITE NAN SCORE LST','COMPOSITE NAN SCORE NDWI','COMPOSITE TIME (sec)']

geegen.exportDataFrame(DRIVE_FOLDER, QA_dict, QAtable_filename, QAtable_columns, OUTDIR_PATHS[3], path2key)
geegen.exportDataFrame(DRIVE_FOLDER, COMP_dict, COMPtable_filename, COMPtable_columns, OUTDIR_PATHS[3], path2key)


06/02/2024 12:10:41 - INFO - 
Exporting image to drive : MODISv21A1D_AQUA_FILT_LST_202311_COMPD2


06/02/2024 12:11:24 - INFO - file_cache is only supported with oauth2client<4.0.0
06/02/2024 12:11:24 - INFO - Attempting refresh to obtain initial access_token
06/02/2024 12:11:25 - INFO - Refreshing access_token
06/02/2024 12:11:26 - INFO - 
Local downloading : MODISv21A1D_AQUA_FILT_LST_202311_COMPD2.tif
06/02/2024 12:11:30 - INFO - 
Exporting table to drive : GEEPREPROC_MODISv21A1D_FILT_New_Caledonia_Fr_20231111_20231120


06/02/2024 12:11:37 - INFO - file_cache is only supported with oauth2client<4.0.0
06/02/2024 12:11:37 - INFO - Attempting refresh to obtain initial access_token
06/02/2024 12:11:37 - INFO - Refreshing access_token
06/02/2024 12:11:39 - INFO - 
Local downloading : GEEPREPROC_MODISv21A1D_FILT_New_Caledonia_Fr_20231111_20231120.csv
06/02/2024 12:11:41 - INFO - 
Exporting table to drive : COUNTCompositeDecade_MODISv21A1D_New_Caledonia_Fr_20231111_20231120


06/02/2024 12:11:49 - INFO - file_cache is only supported with oauth2client<4.0.0
06/02/2024 12:11:49 - INFO - Attempting refresh to obtain initial access_token
06/02/2024 12:11:49 - INFO - Refreshing access_token
06/02/2024 12:11:50 - INFO - 
Local downloading : COUNTCompositeDecade_MODISv21A1D_New_Caledonia_Fr_20231111_20231120.csv
